# Top 3: 166.49599999999998

VNS
 Athletes:
['ondine achampong' 'alice kinsella' 'jessica gadirova'
 'georgia-mae fenton']
Total Score:
167.763



HH
Athletes: ['jessica gadirova' 'alice kinsella' 'georgia-mae fenton'
 'ondine achampong']
Best Score: 166.496



TS
 Athletes:
['georgia-mae fenton' 'jessica gadirova' 'alice kinsella'
 'ondine achampong']
Total Score:
167.796





Brute force
 Athletes:
['alice kinsella', 'georgia-mae fenton', 'jessica gadirova', 'ondine achampong']
Maximum Score:
166.496

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('../../Combine_Data/women/iaa2024.csv')


In [2]:
df = pd.read_csv('../../Combine_Data/women/iaa2024.csv')


In [3]:
df[df['Athlete']=='jessica gadirova']

,Rank,Athlete,Nation,VT,UB,BB,FX,AA,round,year
4,5,jessica gadirova,none,14.400,13.166,12.933,14.100,54.599,qual,2022
181,5,jessica gadirova,none,14.400,13.166,12.933,14.100,54.599,final,2022
412,10,jessica gadirova,none,13.866,12.833,12.233,13.766,52.698,final,2022
576,3,jessica gadirova,none,14.600,13.766,14.000,14.400,56.766,qual,2023


In [4]:
names = [
    "Ondine Achampong",
    "Evie Flage-Donovan",
    "Becky Downie",
    "Ruby Evans",
    "Georgia-Mae Fenton",
    "Claudia Fragapane",
    "Jennifer Gadirova",
    "Jessica Gadirova",
    "Alice Kinsella",
    "Kelly Simm",
    "Ruby Stacey",
    "Poppy-Grace Stickler"
]

lowercase_names = [name.lower() for name in names]

# Display the lowercase names
for lowercase_name in lowercase_names:
    print(lowercase_name)

ondine achampong
evie flage-donovan
becky downie
ruby evans
georgia-mae fenton
claudia fragapane
jennifer gadirova
jessica gadirova
alice kinsella
kelly simm
ruby stacey
poppy-grace stickler


In [5]:

# Create a new DataFrame based on the list of names
filtered_df = df[df['Athlete'].str.lower().isin(lowercase_names)]


In [6]:
df = filtered_df.copy()

In [7]:
df.head(1)

,Rank,Athlete,Nation,VT,UB,BB,FX,AA,round,year
4,5,jessica gadirova,none,14.4,13.166,12.933,14.1,54.599,qual,2022


In [8]:
df = df.drop(columns=['Nation', 'round', 'year', 'AA'])

In [9]:
df.duplicated().sum()

5

In [10]:
df = df.drop_duplicates()

In [11]:
df

,Rank,Athlete,VT,UB,BB,FX
4,5,jessica gadirova,14.400,13.166,12.933,14.100
13,14,alice kinsella,14.266,14.166,13.066,12.300
14,15,ondine achampong,14.466,12.833,12.966,13.333
110,111,georgia-mae fenton,13.900,14.133,12.100,0.000
164,165,jennifer gadirova,0.000,0.000,0.000,13.600
404,2,alice kinsella,13.600,13.966,13.033,13.533
412,10,jessica gadirova,13.866,12.833,12.233,13.766
415,12,ondine achampong,13.933,12.233,13.266,12.666
514,89,georgia-mae fenton,13.466,13.900,0.000,0.000
516,91,jennifer gadirova,0.000,0.000,12.900,13.533


In [12]:
# Group by 'Athlete' and get the index of the maximum score for each apparatus
idx_max = df.groupby('Athlete').idxmax()

# Create a new DataFrame with the highest scores for each athlete
df_max_scores = pd.DataFrame({
    'Athlete': df.loc[idx_max['VT'], 'Athlete'].values,
    'VT': df.loc[idx_max['VT'], 'VT'].values,
    'UB': df.loc[idx_max['UB'], 'UB'].values,
    'BB': df.loc[idx_max['BB'], 'BB'].values,
    'FX': df.loc[idx_max['FX'], 'FX'].values
})


In [13]:
df = df_max_scores.copy()

Variable Neighborhood Search (VNS):


In [14]:
import pandas as pd
import numpy as np


# Define the number of athletes to select
num_athletes_to_select = 4

# Define the apparatuses
apparatuses = ['FX', 'UB', 'BB', 'VT']

# Define the neighborhood size
neighborhood_size = 2  # Adjust as needed

# Initialize the solution randomly
selected_athletes = np.random.choice(df['Athlete'], num_athletes_to_select, replace=False)

# Define a function to calculate the total score for a given solution
def calculate_total_score(selected_athletes):
    selected_scores = df[df['Athlete'].isin(selected_athletes)][apparatuses].values
    top_3_scores = np.sum(np.partition(selected_scores, -3)[:, -3:])
    return top_3_scores

# Define the main VNS algorithm
def variable_neighborhood_search(df, num_athletes_to_select, apparatuses, neighborhood_size, num_iterations):
    best_solution = None
    best_score = float('-inf')

    for iteration in range(num_iterations):
        # Shake: Generate a new solution by randomly changing athletes
        current_solution = np.random.choice(df['Athlete'], num_athletes_to_select, replace=False)
        
        for _ in range(neighborhood_size):
            # Local Search: Optimize the solution locally
            current_score = calculate_total_score(current_solution)
            
            # Perturb: Randomly change some athletes in the current solution
            perturbed_solution = np.random.choice(df['Athlete'], num_athletes_to_select, replace=False)
            perturbed_score = calculate_total_score(perturbed_solution)
            
            # Acceptance Criterion: Accept the perturbed solution if it improves the score
            if perturbed_score > current_score:
                current_solution = perturbed_solution
                current_score = perturbed_score

        # Update the best solution if the current one is better
        if current_score > best_score:
            best_solution = current_solution
            best_score = current_score

    return best_solution, best_score

# Run the VNS algorithm
best_solution, best_score = variable_neighborhood_search(df, num_athletes_to_select, apparatuses, neighborhood_size, num_iterations=1000)

# Print the result
print('Selected Athletes:')
print(best_solution)
print('Total Score:')
print(best_score)


Selected Athletes:
['alice kinsella' 'georgia-mae fenton' 'jessica gadirova'
 'ondine achampong']
Total Score:
167.763


Hyper-Heuristics

In [15]:
import pandas as pd
import numpy as np


# Define the number of athletes to select
num_athletes_to_select = 4

# Define the apparatuses
apparatuses = ['FX', 'VT', 'BB', 'UB']

# Initialize variables to store the best solution
best_score = 0
best_selection = None

# Define the number of iterations for the hyper-heuristic
num_iterations = 1000

for _ in range(num_iterations):
    # Randomly select 4 athletes
    selected_athletes = np.random.choice(df['Athlete'], size=num_athletes_to_select, replace=False)

    # Calculate the score for the current selection
    scores = df[df['Athlete'].isin(selected_athletes)][apparatuses].sum(axis=1).nlargest(3).sum()

    # Update the best solution if the current one is better
    if scores > best_score:
        best_score = scores
        best_selection = selected_athletes

# Print the best selection and score
print("Best Selection:", best_selection)
print("Best Score:", best_score)


Best Selection: ['alice kinsella' 'jessica gadirova' 'ondine achampong' 'ruby evans']
Best Score: 166.496


Tabu Search

In [16]:
import pandas as pd
import numpy as np

def calculate_score(selected_athletes, df):
    """
    Calculate the total score for a given set of selected athletes.
    """
    # Select rows corresponding to the chosen athletes
    selected_df = df[df['Athlete'].isin(selected_athletes)]

    # Calculate the sum of the top 3 scores for each apparatus
    apparatus_scores = selected_df.drop(['Athlete'], axis=1)
    top_3_sum = apparatus_scores.apply(lambda row: sum(sorted(row, reverse=True)[:3]), axis=0)

    # Calculate the total score
    total_score = top_3_sum.sum()

    return total_score




def tabu_search(df, num_athletes_to_select, tabu_size, max_iterations):
    # Initialize the solution with a random set of athletes
    available_athletes = df['Athlete'].tolist()
    current_solution = np.random.choice(available_athletes, size=num_athletes_to_select, replace=False)
    best_solution = current_solution.copy()
    best_score = calculate_score(best_solution, df)

    # Initialize tabu list
    tabu_list = []

    # Main Tabu Search loop
    for iteration in range(max_iterations):
        # Generate neighbors by swapping athletes
        neighbors = []
        for i in range(num_athletes_to_select):
            for athlete in df['Athlete']:
                if athlete not in current_solution:
                    neighbor = current_solution.copy()
                    neighbor[i] = athlete
                    neighbors.append(neighbor)

                    
                    
                best_neighbor = None
        best_neighbor_score = float('-inf')

        for neighbor in neighbors:
            neighbor_score = calculate_score(neighbor, df)

            # Convert the NumPy array to a tuple
            neighbor_tuple = tuple(neighbor)

            if neighbor_score > best_neighbor_score and neighbor_tuple not in tabu_list:
                best_neighbor = neighbor
                best_neighbor_score = neighbor_score

        # Update current solution and tabu list
        current_solution = best_neighbor
        tabu_list.append(tuple(best_neighbor))

        # Keep the tabu list size within the specified limit
        if len(tabu_list) > tabu_size:
            tabu_list.pop(0)

        # Update the best solution if a better one is found
        if best_neighbor_score > best_score:
            best_solution = best_neighbor
            best_score = best_neighbor_score

        # Print progress
        print(f"Iteration {iteration + 1}, Best Score: {best_score}")

    return best_solution, best_score



# Set parameters
num_athletes_to_select = 4
tabu_size = 10
max_iterations = 100

# Perform Tabu Search
best_solution, best_score = tabu_search(df, num_athletes_to_select, tabu_size, max_iterations)

# Print the selected athletes and the corresponding score
print('Selected Athletes:')
print(best_solution)
print('Total Score:')
print(best_score)

      

Iteration 1, Best Score: 167.16299999999998
Iteration 2, Best Score: 167.796
Iteration 3, Best Score: 167.796
Iteration 4, Best Score: 167.796
Iteration 5, Best Score: 167.796
Iteration 6, Best Score: 167.796
Iteration 7, Best Score: 167.796
Iteration 8, Best Score: 167.796
Iteration 9, Best Score: 167.796
Iteration 10, Best Score: 167.796
Iteration 11, Best Score: 167.796
Iteration 12, Best Score: 167.796
Iteration 13, Best Score: 167.796
Iteration 14, Best Score: 167.796
Iteration 15, Best Score: 167.796
Iteration 16, Best Score: 167.796
Iteration 17, Best Score: 167.796
Iteration 18, Best Score: 167.796
Iteration 19, Best Score: 167.796
Iteration 20, Best Score: 167.796
Iteration 21, Best Score: 167.796
Iteration 22, Best Score: 167.796
Iteration 23, Best Score: 167.796
Iteration 24, Best Score: 167.796
Iteration 25, Best Score: 167.796
Iteration 26, Best Score: 167.796
Iteration 27, Best Score: 167.796
Iteration 28, Best Score: 167.796
Iteration 29, Best Score: 167.796
Iteration 30

brute force

In [17]:
import pandas as pd
import itertools
from tqdm import tqdm

num_athletes = 4
apparatuses = ['VT', 'UB', 'BB', 'FX']

# Generate all combinations of athletes
athlete_combinations = list(itertools.combinations(df['Athlete'], num_athletes))

# variables to store maximum score and athletes
max_score = 0
selected_athletes = []

# tqdm progress bar
progress_bar = tqdm(total=len(athlete_combinations))

# Iterate through combination of athletes
for combination in athlete_combinations:
    # Generate all possible combinations of apparatuses for current athlete combination
    apparatus_combinations = list(itertools.permutations(apparatuses, num_athletes))

    # Iterate through each combination of apparatuses
    for apparatus_combination in apparatus_combinations:
        # Calculate the sum of top three scores for current apparatus combination
        score = df[df['Athlete'].isin(combination)][list(apparatus_combination)].sum(axis=1).nlargest(3).sum()

        # Update maximum score and selected athletes if a higher score is found
        if score > max_score:
            max_score = score
            selected_athletes = list(combination)

    # Update progress bar
    progress_bar.update(1)

progress_bar.close()

print('Selected Athletes:')
print(selected_athletes)
print('Maximum Score:')
print(max_score)


100%|███████████████████████████████████████████| 15/15 [00:00<00:00, 23.51it/s]

Selected Athletes:
['alice kinsella', 'georgia-mae fenton', 'jessica gadirova', 'ondine achampong']
Maximum Score:
166.496


In [18]:
import pandas as pd

# Data
data = {
    'Algorithm': ['VNS', 'HH', 'TS', 'Brute Force'],
    'Athletes': [
        ['ondine achampong', 'alice kinsella', 'jessica gadirova', 'georgia-mae fenton'],
        ['jessica gadirova', 'alice kinsella', 'georgia-mae fenton', 'ondine achampong'],
        ['georgia-mae fenton', 'jessica gadirova', 'alice kinsella', 'ondine achampong'],
        ['alice kinsella', 'georgia-mae fenton', 'jessica gadirova', 'ondine achampong']
    ],
    'Score': [167.763, 166.496, 167.796, 166.496]
}

# Create DataFrame
df = pd.DataFrame(data)

df

,Algorithm,Athletes,Score
0,VNS,"[ondine achampong, alice kinsella, jessica gad...",167.763
1,HH,"[jessica gadirova, alice kinsella, georgia-mae...",166.496
2,TS,"[georgia-mae fenton, jessica gadirova, alice k...",167.796
3,Brute Force,"[alice kinsella, georgia-mae fenton, jessica g...",166.496


In [19]:
df['nation']='GBR'

In [20]:
df

,Algorithm,Athletes,Score,nation
0,VNS,"[ondine achampong, alice kinsella, jessica gad...",167.763,GBR
1,HH,"[jessica gadirova, alice kinsella, georgia-mae...",166.496,GBR
2,TS,"[georgia-mae fenton, jessica gadirova, alice k...",167.796,GBR
3,Brute Force,"[alice kinsella, georgia-mae fenton, jessica g...",166.496,GBR


In [21]:
# df.to_csv('great_britain.csv', index=False)

In [42]:
df = pd.read_csv('../../Combine_Data/women/iaa2024.csv')


In [43]:

names = [
    "Ondine Achampong",
    "Evie Flage-Donovan",
    "Becky Downie",
    "Ruby Evans",
    "Georgia-Mae Fenton",
    "Claudia Fragapane",
    "Jennifer Gadirova",
    "Jessica Gadirova",
    "Alice Kinsella",
    "Kelly Simm",
    "Ruby Stacey",
    "Poppy-Grace Stickler"
]

lowercase_names = [name.lower() for name in names]

# Display the lowercase names
for lowercase_name in lowercase_names:
    print(lowercase_name)

ondine achampong
evie flage-donovan
becky downie
ruby evans
georgia-mae fenton
claudia fragapane
jennifer gadirova
jessica gadirova
alice kinsella
kelly simm
ruby stacey
poppy-grace stickler


In [44]:

# Create a new DataFrame based on the list of names
filtered_df = df[df['Athlete'].str.lower().isin(lowercase_names)]


In [45]:
df=filtered_df.copy()

In [46]:
df = df.drop(columns=['Nation', 'round', 'year', 'AA'])


In [47]:
df.duplicated().sum()


5

In [48]:
df = df.drop_duplicates()

In [49]:
idx_max = df.groupby('Athlete').idxmax()

# Create a new DataFrame with the highest scores for each athlete
df_max_scores = pd.DataFrame({
    'Athlete': df.loc[idx_max['VT'], 'Athlete'].values,
    'VT': df.loc[idx_max['VT'], 'VT'].values,
    'UB': df.loc[idx_max['UB'], 'UB'].values,
    'BB': df.loc[idx_max['BB'], 'BB'].values,
    'FX': df.loc[idx_max['FX'], 'FX'].values
})

In [51]:
df=df_max_scores.copy()

In [52]:
df

,Athlete,VT,UB,BB,FX
0,alice kinsella,14.266,14.166,13.433,13.633
1,georgia-mae fenton,13.900,14.133,13.266,0.000
2,jennifer gadirova,0.000,0.000,12.900,13.600
3,jessica gadirova,14.600,13.766,14.000,14.400
4,ondine achampong,14.466,12.833,13.600,13.333
5,ruby evans,13.066,0.000,0.000,13.166


In [53]:
df['AA'] = df.select_dtypes(include='number').sum(axis=1)

In [57]:
df=df.sort_values(by='AA', ascending=False)

In [58]:
df

,Athlete,VT,UB,BB,FX,AA
3,jessica gadirova,14.600,13.766,14.000,14.400,56.766
0,alice kinsella,14.266,14.166,13.433,13.633,55.498
4,ondine achampong,14.466,12.833,13.600,13.333,54.232
1,georgia-mae fenton,13.900,14.133,13.266,0.000,41.299
2,jennifer gadirova,0.000,0.000,12.900,13.600,26.500
5,ruby evans,13.066,0.000,0.000,13.166,26.232


In [59]:
56.766+55.498+54.232

166.49599999999998